In [1]:
import pandas as pd

full_df = pd.read_csv('data/2022_2023/nhl_pbp_20222023.csv')

# full_df = full_df[full_df["Event"].isin(["SHOT", "GOAL"])]

Empty net goal testing

In [2]:
# Trying to find out pulling goalies. In progress
empty_net_condition = (full_df["Away_Goalie"].isnull() & full_df["Away_Goalie_Id"].isnull() & (full_df["Ev_Team"] == full_df["Home_Team"])) | (full_df["Home_Goalie"].isnull() & full_df["Home_Goalie_Id"].isnull() & (full_df["Ev_Team"] == full_df["Away_Team"]))
empty_net_df = full_df[empty_net_condition]
empty_net_goals = empty_net_df[empty_net_df["Event"] == "GOAL"]
# print(empty_net_goals["p1_name"].value_counts())
empty_net_shots = empty_net_df[empty_net_df["Event"] == "SHOT"]  # Note that these are errors. How could there be a shot on net into an empty net that isn't a goal?
print(len(empty_net_goals))
# Line 7 should match https://www.statmuse.com/nhl/ask?q=teams+with+least+empty+net+goals+2022-2023 and  https://www.statmuse.com/nhl/ask?q=teams+with+most+empty+net+goals+2022-2023 totals

print((len(empty_net_shots)))  # NOTE: These come very close to too many men penalties, so the goalie isn't pulled but there are 6 on?

463
2


Penalty Shot Testing

In [3]:
penalty_full_df = full_df[full_df['Description'].str.contains('Penalty Shot')]
len(penalty_full_df)
# NOTE: Include Shootout Shots? Should penalty shots even be included at all?

55

The block below calculates the time elapsed in seconds, from when the event team had its last shot (including misses.)

In [4]:
shot_df = full_df[full_df["Event"].isin(["SHOT", "GOAL", "MISS"])]

# Convert elapsed_time to timedelta for calculations
shot_df['Seconds_Elapsed'] = pd.to_timedelta(shot_df['Seconds_Elapsed'], unit='s')

# Sort the dataframe by game_id, period, and elapsed_time
# NOTE: Isn't this already done?
shot_df = shot_df.sort_values(by=['Game_Id', 'Period', 'Seconds_Elapsed'])

# Create a new column to store the time since last event or start of period
shot_df['Time_Since_Last_Shot'] = shot_df.groupby(['Game_Id', 'Period', 'Ev_Team'])['Seconds_Elapsed'].diff().fillna(shot_df['Seconds_Elapsed'])

print(shot_df['Time_Since_Last_Shot'].value_counts())

# TODO: Time since last shot for the non-event team? (Would show transition, etc.) 
	# This just shows time since last shot, rather than having time since last event + the prev event location

Time_Since_Last_Shot
0 days 00:00:01    3368
0 days 00:00:02    2917
0 days 00:00:07    2294
0 days 00:00:06    2269
0 days 00:00:05    2220
                   ... 
0 days 00:16:52       1
0 days 00:09:32       1
0 days 00:09:41       1
0 days 00:09:12       1
0 days 00:10:29       1
Name: count, Length: 706, dtype: int64


C:\Users\agjri\AppData\Local\Temp\ipykernel_25408\2366518473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['Seconds_Elapsed'] = pd.to_timedelta(shot_df['Seconds_Elapsed'], unit='s')


Shot Location Adjustments

In [5]:
# TODO: (If a shot comes from the blue line, how can you tell which way it's going?). Use home_players and away_players?